<a href="https://colab.research.google.com/github/Nawel-Bellil/intelligent-travel-assistant-system-Llama/blob/main/notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

![](./cover.jpg)

Your airline's customer service team has been collecting chat data for years—thousands of conversations, each labeled with the user’s intent and an ideal response. Now, it's time to put that data to work.

You've been tasked with fine-tuning a TinyLlama model to power the airline’s next-gen AI assistant. The goal? Given a user message, the model should predict the intent (like booking a flight, checking baggage status, or requesting special assistance) and generate a helpful, human-like response. Accurate intent detection is key since it helps the system understand what the customer wants, so it can respond appropriately and trigger downstream actions when needed.

### The Data
You'll work with a dataset of various travel query examples.

 Column | Description |
|--------|-------------|
| ```instruction``` | A user request from the Travel domain |
| ```category``` | The high-level semantic category for the intent |
| ```intent``` | The specific intent corresponding to the user instruction |
| ```response``` | An example of an expected response from the virtual assistant |

___
### Update to Python 3.10

Due to how frequently the libraries required for this project are updated, you'll need to update your environment to Python 3.10:

1. In the workbook, click on "Environment," in the top toolbar and select "Session details".

2. In the workbook language dropdown, select "Python 3.10".

3. Click "Confirm" and hit "Done" once the session is ready.

In [1]:
# First install the necessary packages
!pip install -q -q -q trl==0.16.0
!pip install -q -q -q tf-keras==2.19.0
!pip install -q -q -q peft==0.14.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 335.7/335.7 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 18.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 41.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 23.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 38.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
# Import the required dependencies for this project
from transformers import AutoTokenizer, AutoModelForCausalLM
from trl import SFTTrainer, SFTConfig
from peft import LoraConfig

from datasets import Dataset, load_dataset
from collections import Counter, defaultdict
import random
import transformers
import torch

The code below loads the travel query dataset and reduces it from ~30k to ~50 records, keeping all intent types. This speeds up fine-tuning. Run it before starting, and feel free to experiment with it later!

In [3]:
# First load the entire dataset
ds = load_dataset('bitext/Bitext-travel-llm-chatbot-training-dataset', split="train")

# Group examples by intent
random.seed(42)
intent_groups = defaultdict(list)
for record in ds:
    intent = record["intent"]
    intent_groups[intent].append(record)

# Determine how many samples per intent
total_intents = len(intent_groups)
samples_per_intent = 100 // total_intents

# Sample from each intent
balanced_subset = []
for intent, examples in intent_groups.items():
    sampled = random.sample(examples, min(samples_per_intent, len(examples)))
    balanced_subset.extend(sampled)

total_num_of_records = 50
travel_chat_ds = Dataset.from_list(balanced_subset[:total_num_of_records])

travel_chat_ds.to_pandas().head(3)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/9.21k [00:00<?, ?B/s]

(…)-travel-llm-chatbot-training-dataset.csv:   0%|          | 0.00/19.7M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/31658 [00:00<?, ? examples/s]

,instruction,intent,category,tags,response
0,I'd like information about my checked baggage ...,check_baggage_allowance,BAGGAGE,BCIP,To retrieve your checked baggage allowance det...
1,i have to see the fucking checked baggage allo...,check_baggage_allowance,BAGGAGE,BCIQW,"To determine your checked baggage allowance, p..."
2,I want to know about my checked baggage allowa...,check_baggage_allowance,BAGGAGE,BCI,To find details regarding your checked baggage...


In [4]:
# Start the project with the dataset below

display(travel_chat_ds)

Dataset({
    features: ['instruction', 'intent', 'category', 'tags', 'response'],
    num_rows: 50
})

In [5]:
# Start coding here
# Use as many cells as you need

In [6]:

model = "PY007/TinyLlama-1.1B-Chat-v0.1"
tokenizer = AutoTokenizer.from_pretrained(model)
pipeline = transformers.pipeline(
    "text-generation",
    model=model,
    torch_dtype=torch.float16,
    device_map="auto",
)

prompt = "What are the values in open source projects?"
formatted_prompt = (
    f"### Human: {prompt}### Assistant:"
)


sequences = pipeline(
    formatted_prompt,
    do_sample=True,
    top_k=50,
    top_p = 0.7,
    num_return_sequences=1,
    repetition_penalty=1.1,
    max_new_tokens=500,
)
for seq in sequences:
    print(f"Result: {seq['generated_text']}")


tokenizer_config.json:   0%|          | 0.00/762 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/21.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/438 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/652 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/4.40G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/63.0 [00:00<?, ?B/s]

Device set to use cuda:0


Result: ### Human: What are the values in open source projects?### Assistant: There are many values that make up an open source project. Some of the most common values include:

Freedom to change - An open source project should have a clear statement of what it is capable of, and the community can freely modify or remove parts of the code without any fear of legal action.

License - Open source projects should use licenses that allow users to do whatever they want with the code. This helps ensure that the project remains open and accessible, even after it's been bought by a larger company.

Transparency - Open source projects should be transparent about their development process and the role of each person involved. This helps to maintain trust between the developers and the wider community.

Security - Open source projects should also be secure, using secure coding practices and regularly audited by security experts.

Free-to-use - Many open source projects are free to use, but some a